# Create databases

This will create databases from root schema for testing clearscape analytics

Database hierarchy is
CS01
CS01_RAW
CS01_RAW_VIEW
CS01_TARGET
CS01_TARGET_VIEW

In [2]:
import teradatasql
import yaml
import sys
import os
import sqlparse
import pandas as pd



In [3]:
# get home directory
from os.path import expanduser
import pprint
home = expanduser("~")
# assume project is called dbt_clearscape
project = 'dbt_clearscape'
# read config file
with open(home + '/.dbt/profiles.yml') as file:
    config = yaml.load(file, Loader=yaml.FullLoader)
# get project config
if project not in config:
    print('project not found in config file')
project_config = config[project]
# get target
target = project_config['target']
# get connection details
project_connection = project_config['outputs'][target]
#pprint.pprint(project_connection)


In [4]:
# connect to teradata
conn = teradatasql.connect(host=project_connection['server'], user=project_connection['username'], password=project_connection['password'])    

Connect to database

In [5]:
# connect to teradata
conn = teradatasql.connect(host=project_connection['server'], user=project_connection['username'], password=project_connection['password'])    

# Create all the database

The databases are

- CS01
- CS01_RAW
- CS01_RAW_VIEW
- CS01_TARGET
- CS01_TARGET_VIEW



In [6]:
# TODO - setup parameterised db config
db_list = {}
db_list ['CS01'] = {'owner': {project_connection['username']},'space':0}
db_list ['CS01_RAW'] = {'owner': 'CS01','space':0}
db_list ['CS01_RAW_VIEW'] = {'owner': 'CS01','space':0}
db_list ['CS01_TARGET'] = {'owner': 'CS01','space':10000}
db_list ['CS01_TARGET_VIEW'] = {'owner': 'CS01','space':0}

In [ ]:
# Delete and then drop all databases
with conn.cursor() as csr:
    for i,db in enumerate(db_list):
        # It is sometime very slow in deleting databases so drop each table and view first
        with conn.cursor() as csr2:
            csr2.execute(f"""SELECT tablename FROM DBC.tablesv WHERE DATABASENAME = '{db}' AND tablekind in ('T','O')""")
            # copy list of tables in table_list
            table_list = csr2.fetchall()
            for table in table_list:
                print(f'DROP TABLE {db}.{table[0]}')
                csr2.execute(f'DROP TABLE {db}.{table[0]}')
            csr2.execute(f"""SELECT tablename FROM DBC.tablesv WHERE DATABASENAME = '{db}' AND tablekind in ('V')""")
            # copy list of views in view_list
            view_list = csr2.fetchall()
            for view in view_list:
                print(f'DROP VIEW {db}.{view[0]}')
                csr2.execute(f'DROP VIEW {db}.{view[0]}')
        print(i,db)
        print(f'DELETE DATABASE {db}')
        csr.execute(f'DELETE DATABASE {db}')
        print(f'GIVE {db} TO {project_connection["username"]}')
        csr.execute(f'GIVE {db} TO {project_connection['username']}')
with conn.cursor() as csr:
    for i,db in enumerate(db_list):
        print(i,db)
        print(f'DROP DATABASE {db}')
        csr.execute(f'DROP DATABASE {db}')

In [25]:
# TODO : Convert to a more parameteris version

In [ ]:
sql = f"CREATE DATABASE CS01 FROM {project_connection['username']} AS PERM=0"
with conn.cursor() as cur:
    cur.execute(sql, ignoreErrors=[5612])
    print('Database created')
    cur.execute("COMMENT ON CS01 AS 'Base database for Clearscape'")
    print('Comment added')

In [ ]:
sql = f"CREATE DATABASE CS01_RAW FROM CS01 AS PERM=0"
with conn.cursor() as cur:
    cur.execute(sql, ignoreErrors=[5612])
    print('Database created')
    cur.execute("COMMENT ON CS01_RAW AS 'raw database for Clearscape'")
    print('Comment added')

In [ ]:
sql = f"CREATE DATABASE CS01_RAW_VIEW FROM CS01 AS PERM=0"
with conn.cursor() as cur:
    cur.execute(sql, ignoreErrors=[5612])
    print('Database created')
    cur.execute("COMMENT ON CS01_RAW_VIEW AS 'raw database for Clearscape'")
    print('Comment added')

In [ ]:
sql = f"CREATE DATABASE CS01_TARGET FROM CS01 AS PERM=0"
with conn.cursor() as cur:
    cur.execute(sql, ignoreErrors=[5612])
    print('Database created')
    cur.execute("COMMENT ON CS01_TARGET AS 'target database for Clearscape'")
    print('Comment added')

In [ ]:
sql = f"CREATE DATABASE CS01_TARGET_VIEW FROM CS01 AS PERM=0"
with conn.cursor() as cur:
    cur.execute(sql, ignoreErrors=[5612])
    print('Database created')
    cur.execute("COMMENT ON CS01_TARGET_VIEW AS 'target database for Clearscape'")
    print('Comment added')

In [ ]:
# grant intra database access
sql = f"GRANT SELECT, INSERT, UPDATE, DELETE ON CS01_RAW TO CS01_RAW_VIEW WITH GRANT OPTION"
with conn.cursor() as cur:
    cur.execute(sql)
    print('Grant created')

sql = f"GRANT SELECT, INSERT, UPDATE, DELETE ON CS01_TARGET TO CS01_TARGET_VIEW WITH GRANT OPTION"
with conn.cursor() as cur:
    cur.execute(sql)
    print('Grant created')

sql = f"GRANT SELECT, INSERT, UPDATE, DELETE ON CS01_RAW_VIEW TO CS01_TARGET_VIEW WITH GRANT OPTION"
with conn.cursor() as cur:
    cur.execute(sql)
    print('Grant created')

# dbc access
sql = f"GRANT SELECT ON DBC TO CS01_RAW _VIEW WITH GRANT OPTION"
with conn.cursor() as cur:
    cur.execute(sql)
    print('Grant created')


# val access
sql = f"GRANT SELECT ON VAL TO CS01_RAW _VIEW WITH GRANT OPTION"
with conn.cursor() as cur:
    cur.execute(sql)
    print('Grant created')


Allocate space for the databases

In [32]:
with conn.cursor() as csr:
    # drop space database if it exists
    csr.execute('DROP DATABASE CS01_SPACE',ignoreErrors = [3802])
    # create space data with space
    csr.execute(f'CREATE DATABASE CS01_SPACE FROM {project_connection['username']}  AS PERM=100000000 ')
    # give space to database
    csr.execute(f'GIVE CS01_SPACE TO CS01_RAW')
    # drop space
    csr.execute('DROP DATABASE CS01_SPACE',ignoreErrors = [])

In [33]:
with conn.cursor() as csr:
    # drop space database if it exists
    csr.execute('DROP DATABASE CS01_SPACE',ignoreErrors = [3802])
    # create space data with space
    csr.execute(f'CREATE DATABASE CS01_SPACE FROM {project_connection['username']}  AS PERM=100000000 ')
    # give space to database
    csr.execute(f'GIVE CS01_SPACE TO CS01_TARGET')
    # drop space
    csr.execute('DROP DATABASE CS01_SPACE',ignoreErrors = [])